In [1]:
from thesis import *

ModuleNotFoundError: No module named 'transformers'

In [ ]:
EmbeddingModel.set_model("gpt2")

In [3]:
EmbeddingModel.embed("This is a test sentence.").shape

(1, 768)

In [4]:
model = ModelRunner()

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [5]:
model.to_device()

In [6]:
model.set_optimizer()

In [ ]:
def embedding_distances(dataset):
        emb = embed_column(dataset)
        centers = find_clusters(emb, n_clusters=5)
        distances = [min([np.linalg.norm(e - c) for c in centers]) for e in emb]
        dataset = dataset.add_column("distances", distances)
        dataset = dataset.add_column("mean",np.mean(distances))
        return dataset

In [ ]:
model.set_dataset(load_dataset("SQuAD"), args = {"mask_train":embedding_distances})

Map: 100%|██████████| 10570/10570 [00:01<00:00, 5704.13 examples/s]


In [ ]:
def update_lr(batch, epoch,optimizer,range_coef=1.7, aim=5e-5, expected_lr=5e-5, goalr=5.0):
    av = batch["distances"]
    # Adjust LR (with clamp)
    #balanced = 5e-5 - (range_coef*(av))*(5e-5)
    balanced = range_coef*((av/batch["mean"])-1) + 5e-5
    #new_lr = balanced*((expected_lr)/(goalr))
    new_lr = max(1e-6, min(1e-4, balanced))
    print(balanced,range_coef)
    for param_group in optimizer.param_groups:
        param_group['lr'] = new_lr



In [ ]:

    
model.train(update=update_lr, epochs=1)
        

In [ ]:
model.evaluate()

In [ ]:
# Distance to cluster means
        #distances = [np.linalg.norm(emb - cm) for cm in cluster_means]
        #av = min(distances)



In [9]:
model.train()

d:\THESIS\thesis-1\thesis\run.py:37: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
d:\Soft\CONDA\envs\MAIN\Lib\site-packages\torch\amp\grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
Epoch 1:   0%|          | 0/2000 [00:00<?, ?it/s]

d:\Soft\CONDA\envs\MAIN\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
d:\THESIS\thesis-1\thesis\run.py:48: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


KeyboardInterrupt: 

In [ ]:
model.evaluate()

In [ ]:

# Training loop
for epoch in range(3):
    print(f"Epoch {epoch + 1}")
    for step, batch in enumerate(train_loader):
        # Get precomputed embedding for this sample
        emb = context_embeddings[step]

        # Distance to cluster means
        #distances = [np.linalg.norm(emb - cm) for cm in cluster_means]
        #av = min(distances)
        av = distances[step]
        # Adjust LR (with clamp)
        #balanced = 5e-5 - (range_coef*(av))*(5e-5)


        balanced = range_coef*((av/aim)-1) + 5e-5

        #new_lr = balanced*((expected_lr)/(goalr))
        new_lr = max(1e-6, min(1e-4, balanced))
        print(balanced,range_coef)
        for param_group in optimizer.param_groups:
            param_group['lr'] = new_lr

        # Forward/backward
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if step % 10 == 0:
            print(f"Step {step}, Loss: {loss.item():.4f}, LR: {new_lr:.6f}")

